In [ ]:
# LANGSMITH_TRACING=true
# LANGSMITH_ENDPOINT="<langsmith-endpoint>"
# LANGSMITH_API_KEY="<your-langsmith-api-key>"
# LANGSMITH_PROJECT="<your-langsmith-project>"
# OPENAI_API_KEY="<your-openai-api-key>"

In [ ]:
import getpass, os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

In [ ]:
_set_env("OPENAI_API_KEY")

In [ ]:
_set_env("LANGSMITH_API_KEY")

In [ ]:
from typing import Literal
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, RemoveMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState, StateGraph, START, END

In [ ]:
memory = MemorySaver()

# Add 'summary' attribute.
class State(MessagesState):
    summary: str

model = ChatOpenAI(model="gpt-4o-mini")

def call_model(state: State):
    # If a summary exists, add this in as a system message.
    summary = state.get("summary", "")
    if summary:
        system_message = f"Summary of conversation earlier: {summary}"
        messages = [SystemMessage(content=system_message)] + state["messages"]
    else:
        messages = state["messages"]
    response = model.invoke(messages)

    # We return a list. This will get added to the existing list.
    return {"messages": [response]}

In [ ]:
max_msgs = 6
def should_continue(state: State) -> Literal["summarize_conversation", END]:
    """Return the next node to execute."""
    messages = state["messages"]
    # If there are more than max_msgs messages, summarize the conversation.
    if len(messages) > max_msgs:
        return "summarize_conversation"
    else:
        return END


In [ ]:
def summarize_conversation(state: State):
    # First, summarize the conversation.
    summary = state.get("summary", "")
    if summary:
        # If a summary already exists, we use a different system prompt to
        # summarize it than if one didn't exist.
        summary_message = (
            f"This is a summary of the conversation to date: {summary}\n\n "
            "Extend the summary by taking into account the new messages above:"
        )
    else:
        summary_message = "Create a summary of the conversation above:"

    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = model.invoke(messages)

    # We now need to delete messages that we no longer want to show up.
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:2]]

    return {"summary": response.content, "messages": delete_messages}

In [ ]:
# Define a new graph.
workflow = StateGraph(State)

# Define the conversation node and the summarize node.
workflow.add_node("conversation", call_model)
workflow.add_node(summarize_conversation)

# Set the entrypoint as conversation.
workflow.add_edge(START, "conversation")

# Add a conditional edge.
workflow.add_conditional_edges(
    # Define the start node with "conversation".
    "conversation",
    # And then pass in the function that determines which node is called next.
    should_continue,
)

# We now add a normal edge from 'summarize_conversation' to END.
workflow.add_edge("summarize_conversation", END)

# ...compile!
app = workflow.compile(checkpointer=memory)

In [ ]:
def print_update(update):
    for k, v in update.items():
        for m in v["messages"]:
            m.pretty_print()
        if "summary" in v:
            print(v["summary"])

In [ ]:
config = {"configurable": {"thread_id": "4"}}
input_message = HumanMessage(content="Hi! I'm Dan")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

input_message = HumanMessage(content="what's my name?")
input_message .pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

input_message = HumanMessage(content="I like the Eagles!")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)